# PISA Data Description and Wrangling

## Overview

PISA is a survey of students' skills and knowledge as they approach the end of compulsory education. It is not a conventional school test. Rather than examining how well students have learned the school curriculum, it looks at how well prepared they are for life beyond school.

Around 510,000 students in 65 economies took part in the PISA 2012 assessment of reading, mathematics and science representing about 28 million 15-year-olds globally. Of those economies, 44 took part in an assessment of creative problem solving and 18 in an assessment of financial literacy. For more details see [PISA website](https://www.oecd.org/pisa/pisaproducts/datavisualizationcontest.htm).

The data and topics of investigation come from the PISA Data Visualization Competition.

#### Topics suggested by Udacity:

1. The importance of school factors in explaining academic performance.

2. **Differences in achievement based on gender, location, or student attitudes.**

3. Differences in achievement based on teacher practices and attitudes.

4. Inequalities in academic achievement.

## Data wrangling

In this report the PISA 2012 will be used to investigate the differences in achievement in mathematics tests based on location, gender and student attitudes. Keeping these tasks in mind, the data wrangling will proceed as follows:

1. Download the two datafiles 'pisadict2012.csv' (which contains the description of all codes and abbreviations in the main table) and 'pisa2012.csv' (the main datafile, the unzipped csv file is 2.75 GB).

2. Wrangle the dictionary of terms file, keep only those columns that are relevant to this analysis.

3. Use sqlalchemy to extract a managable size Pandas dataframe from the main PISA data file, this is done using tthe methods described in [Working with large csv files in Python](https://pythondata.com/working-large-csv-files-python/).

4. Clean some minor issues regarding the countries involved in the study.

In [1]:
### import the necessary packages to work with the datasets  
import numpy as np
import pandas as pd

from sqlalchemy import create_engine

In [2]:
### option to display full content of columns in the dataframes
pd.set_option('display.max_colwidth', -1)

### The dictionary of terms datafile

In [3]:
### save the dictionary of terms as pandas dataframe
df_dict=pd.read_csv("pisadict2012.csv", encoding='iso-8859-1')

In [4]:
### investigate the dataframe
df_dict.sample(4)

,Unnamed: 0,x
585,W_FSTR35,FINAL STUDENT REPLICATE BRR-FAY WEIGHT35
442,HOSTCUL,Acculturation: Host Culture Oriented Strategies
551,W_FSTR1,FINAL STUDENT REPLICATE BRR-FAY WEIGHT1
254,ST89Q05,Attitude toward School - Trying Hard is Important


In [5]:
### rename the columns
df_dict.columns = ['Code', 'Description']

In [6]:
### get more information about the dataframe
df_dict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 635 entries, 0 to 634
Data columns (total 2 columns):
Code           635 non-null object
Description    635 non-null object
dtypes: object(2)
memory usage: 10.0+ KB


#### Downsizing the list of columns to be used in the analysis:
1. Investigate the available codes in the list, there are several categories of such codes: abbreviations, ST#, IC#, EC#, PV# and W_FSTR#. 
2. Look at ST# codes, identification codes from the beginning of the dataframe, and codes that contain the work mathematics (or versions of it) in description.
3. ST codes are divided on categories, keep some of ST3 and all ST4 and ST9 entries.
4. Manually select the codes that make reference to mathematics in description.

In [7]:
df_dict.head(20);

In [8]:
### the codes that contain information about the student
df1 = df_dict.iloc[[0, 1, 2, 3, 6, 7, 11], :]
df1

,Code,Description
0,CNT,Country code 3-character
1,SUBNATIO,Adjudicated sub-region code 7-digit code (3-digit country code + region ID + stratum ID)
2,STRATUM,Stratum ID 7-character (cnt + region ID + original stratum ID)
3,OECD,OECD country
6,STIDSTD,Student ID
7,ST01Q01,International Grade
11,ST04Q01,Gender


In [9]:
### investigate the ST0# codes and extract them in a separate dataframe
df_st0=df_dict[df_dict['Code'].str.contains('ST0')]
df_st0

,Code,Description
7,ST01Q01,International Grade
8,ST02Q01,National Study Programme
9,ST03Q01,Birth - Month
10,ST03Q02,Birth -Year
11,ST04Q01,Gender
12,ST05Q01,Attend <ISCED 0>
13,ST06Q01,Age at <ISCED 1>
14,ST07Q01,Repeat - <ISCED 1>
15,ST07Q02,Repeat - <ISCED 2>
16,ST07Q03,Repeat - <ISCED 3>


In [10]:
### investigate the ST1# codes and extract them in a separate dataframe
df_st1=df_dict[df_dict['Code'].str.contains('ST1')]
df_st1

,Code,Description
19,ST115Q01,Truancy - Skip classes within school day
20,ST11Q01,At Home - Mother
21,ST11Q02,At Home - Father
22,ST11Q03,At Home - Brothers
23,ST11Q04,At Home - Sisters
24,ST11Q05,At Home - Grandparents
25,ST11Q06,At Home - Others
26,ST13Q01,Mother<Highest Schooling>
27,ST14Q01,Mother Qualifications - <ISCED level 6>
28,ST14Q02,Mother Qualifications - <ISCED level 5A>


In [11]:
### investigate the ST2# codes and extract them in a separate dataframe
df_st2=df_dict[df_dict['Code'].str.contains('ST2')]
df_st2

,Code,Description
38,ST20Q01,Country of Birth International - Self
39,ST20Q02,Country of Birth International - Mother
40,ST20Q03,Country of Birth International - Father
41,ST21Q01,Age of arrival in <country of test>
42,ST25Q01,International Language at Home
43,ST26Q01,Possessions - desk
44,ST26Q02,Possessions - own room
45,ST26Q03,Possessions - study place
46,ST26Q04,Possessions - computer
47,ST26Q05,Possessions - software


In [12]:
### investigate the ST3# codes and extract them in a separate dataframe
df_st3_all=df_dict[df_dict['Code'].str.contains('ST3')]
df_st3_all

,Code,Description
74,ST35Q01,Subjective Norms -Friends Do Well in Mathematics
75,ST35Q02,Subjective Norms -Friends Work Hard on Mathematics
76,ST35Q03,Subjective Norms - Friends Enjoy Mathematics Tests
77,ST35Q04,Subjective Norms - Parents Believe Studying Mathematics Is Important
78,ST35Q05,Subjective Norms - Parents Believe Mathematics Is Important for Career
79,ST35Q06,Subjective Norms - Parents Like Mathematics
80,ST37Q01,Math Self-Efficacy - Using a <Train Timetable>
81,ST37Q02,Math Self-Efficacy - Calculating TV Discount
82,ST37Q03,Math Self-Efficacy - Calculating Square Metres of Tiles
83,ST37Q04,Math Self-Efficacy - Understanding Graphs in Newspapers


In [13]:
### only the first six ST3# codes are relevant for this study
### extract them in a separate dataframe
df_st3 = df_st3_all.iloc[0:6, :]
df_st3

,Code,Description
74,ST35Q01,Subjective Norms -Friends Do Well in Mathematics
75,ST35Q02,Subjective Norms -Friends Work Hard on Mathematics
76,ST35Q03,Subjective Norms - Friends Enjoy Mathematics Tests
77,ST35Q04,Subjective Norms - Parents Believe Studying Mathematics Is Important
78,ST35Q05,Subjective Norms - Parents Believe Mathematics Is Important for Career
79,ST35Q06,Subjective Norms - Parents Like Mathematics


In [14]:
### investigate the ST4# codes and extract them in a separate dataframe
df_st4=df_dict[df_dict['Code'].str.contains('ST4')]
df_st4

,Code,Description
88,ST42Q01,Math Anxiety - Worry That It Will Be Difficult
89,ST42Q02,Math Self-Concept - Not Good at Maths
90,ST42Q03,Math Anxiety - Get Very Tense
91,ST42Q04,Math Self-Concept- Get Good <Grades>
92,ST42Q05,Math Anxiety - Get Very Nervous
93,ST42Q06,Math Self-Concept - Learn Quickly
94,ST42Q07,Math Self-Concept - One of Best Subjects
95,ST42Q08,Math Anxiety - Feel Helpless
96,ST42Q09,Math Self-Concept - Understand Difficult Work
97,ST42Q10,Math Anxiety - Worry About Getting Poor <Grades>


In [15]:
### investigate the ST5# codes and extract them in a separate dataframe
df_st5=df_dict[df_dict['Code'].str.contains('ST5')]
df_st5

,Code,Description
132,ST53Q01,Learning Strategies- Important Parts vs. Existing Knowledge vs. Learn by Heart
133,ST53Q02,Learning Strategies- Improve Understanding vs. New Ways vs. Memory
134,ST53Q03,Learning Strategies - Other Subjects vs. Learning Goals vs. Rehearse Problems
135,ST53Q04,Learning Strategies - Repeat Examples vs. Everyday Applications vs. More Information
136,ST55Q01,Out of school lessons - <test lang>
137,ST55Q02,Out of school lessons - <maths>
138,ST55Q03,Out of school lessons - <science>
139,ST55Q04,Out of school lessons - other
140,ST57Q01,Out-of-School Study Time - Homework
141,ST57Q02,Out-of-School Study Time - Guided Homework


In [16]:
### investigate the ST6# codes and extract them in a separate dataframe
df_st6=df_dict[df_dict['Code'].str.contains('ST6')]
df_st6

,Code,Description
146,ST61Q01,Experience with Applied Maths Tasks - Use <Train Timetable>
147,ST61Q02,Experience with Applied Maths Tasks - Calculate Price including Tax
148,ST61Q03,Experience with Applied Maths Tasks - Calculate Square Metres
149,ST61Q04,Experience with Applied Maths Tasks - Understand Scientific Tables
150,ST61Q05,Experience with Pure Maths Tasks - Solve Equation 1
151,ST61Q06,Experience with Applied Maths Tasks - Use a Map to Calculate Distance
152,ST61Q07,Experience with Pure Maths Tasks - Solve Equation 2
153,ST61Q08,Experience with Applied Maths Tasks - Calculate Power Consumption Rate
154,ST61Q09,Experience with Applied Maths Tasks - Solve Equation 3
155,ST62Q01,Familiarity with Math Concepts - Exponential Function


In [17]:
### investigate the ST7# codes and extract them in a separate dataframe
df_st7=df_dict[df_dict['Code'].str.contains('ST7')]
df_st7

,Code,Description
174,ST70Q01,No of <class period> p/wk - <test lang>
175,ST70Q02,No of <class period> p/wk - <Maths>
176,ST70Q03,No of <class period> p/wk - <Science>
177,ST71Q01,No of ALL <class period> a week
178,ST72Q01,Class Size - No of Students in <Test Language> Class
179,ST73Q01,OTL - Algebraic Word Problem in Math Lesson
180,ST73Q02,OTL - Algebraic Word Problem in Tests
181,ST74Q01,OTL - Procedural Task in Math Lesson
182,ST74Q02,OTL - Procedural Task in Tests
183,ST75Q01,OTL - Pure Math Reasoning in Math Lesson


In [18]:
### investigate the ST8# codes and extract them in a separate dataframe
df_st8=df_dict[df_dict['Code'].str.contains('ST8')]
df_st8

,Code,Description
205,ST80Q01,Cognitive Activation - Teacher Encourages to Reflect Problems
206,ST80Q04,Cognitive Activation - Gives Problems that Require to Think
207,ST80Q05,Cognitive Activation - Asks to Use Own Procedures
208,ST80Q06,Cognitive Activation - Presents Problems with No Obvious Solutions
209,ST80Q07,Cognitive Activation - Presents Problems in Different Contexts
210,ST80Q08,Cognitive Activation - Helps Learn from Mistakes
211,ST80Q09,Cognitive Activation - Asks for Explanations
212,ST80Q10,Cognitive Activation - Apply What We Learned
213,ST80Q11,Cognitive Activation - Problems with Multiple Solutions
214,ST81Q01,Disciplinary Climate - Students Dont Listen


In [19]:
### investigate the ST9# codes and extract them in a separate dataframe
df_st9=df_dict[df_dict['Code'].str.contains('ST9')]
df_st9

,Code,Description
255,ST91Q01,Perceived Control - Can Succeed with Enough Effort
256,ST91Q02,Perceived Control - My Choice Whether I Will Be Good
257,ST91Q03,Perceived Control - Problems Prevent from Putting Effort into School
258,ST91Q04,Perceived Control - Different Teachers Would Make Me Try Harder
259,ST91Q05,Perceived Control - Could Perform Well if I Wanted
260,ST91Q06,Perceived Control - Perform Poor Regardless
261,ST93Q01,Perseverance - Give up easily
262,ST93Q03,Perseverance - Put off difficult problems
263,ST93Q04,Perseverance - Remain interested
264,ST93Q06,Perseverance - Continue to perfection


In [20]:
### create a list of codes that contain the word M(m)athematic(s) in their description
### and are not in the ST# category

df_math = df_dict[(df_dict['Description'].str.contains('athematic'))
                 & (~df_dict['Code'].str.contains('ST3'))
                 & (~df_dict['Code'].str.contains('ST4'))]
df_math

,Code,Description
413,ANXMAT,Mathematics Anxiety
419,CLSMAN,Mathematics Teacher's Classroom Management
423,COGACT,Cognitive Activation in Mathematics Lessons
429,EXAPPLM,Experience with Applied Mathematics Tasks at School
430,EXPUREM,Experience with Pure Mathematics Tasks at School
431,FAILMAT,Attributions to Failure in Mathematics
432,FAMCON,Familiarity with Mathematical Concepts
433,FAMCONC,Familiarity with Mathematical Concepts (Signal Detection Adjusted)
452,INSTMOT,Instrumental Motivation for Mathematics
453,INTMAT,Mathematics Interest


In [21]:
### create a list of codes that contains reference to math 
df_math_mat = df_math[(df_math['Code'].str.contains('MAT'))
                     & (~df_math['Code'].str.contains('ANC'))]
df_math_mat

,Code,Description
413,ANXMAT,Mathematics Anxiety
431,FAILMAT,Attributions to Failure in Mathematics
453,INTMAT,Mathematics Interest
461,MATBEH,Mathematics Behaviour
462,MATHEFF,Mathematics Self-Efficacy
463,MATINTFC,Mathematics Intentions
464,MATWKETH,Mathematics Work Ethic
475,SCMAT,Mathematics Self-Concept
485,USEMATH,Use of ICT in Mathematic Lessons
500,PV1MATH,Plausible value 1 in mathematics


In [22]:
### use the previous steps to create a dataframe 
### that contains the list of column names to be extracted from the main datafile
df_dict_clean=pd.concat([df1, df_st3, df_st4, df_st9, df_math_mat])
df_dict_clean

,Code,Description
0,CNT,Country code 3-character
1,SUBNATIO,Adjudicated sub-region code 7-digit code (3-digit country code + region ID + stratum ID)
2,STRATUM,Stratum ID 7-character (cnt + region ID + original stratum ID)
3,OECD,OECD country
6,STIDSTD,Student ID
7,ST01Q01,International Grade
11,ST04Q01,Gender
74,ST35Q01,Subjective Norms -Friends Do Well in Mathematics
75,ST35Q02,Subjective Norms -Friends Work Hard on Mathematics
76,ST35Q03,Subjective Norms - Friends Enjoy Mathematics Tests


In [23]:
### store the selected set of codes as a csv file
df_dict_clean.to_csv('pisadict2012_clean.csv', index=False)

In [24]:
### write the selected codes to a list and print this list
selected_codes = df_dict_clean['Code'].tolist()
print(','.join(selected_codes))


CNT,SUBNATIO,STRATUM,OECD,STIDSTD,ST01Q01,ST04Q01,ST35Q01,ST35Q02,ST35Q03,ST35Q04,ST35Q05,ST35Q06,ST42Q01,ST42Q02,ST42Q03,ST42Q04,ST42Q05,ST42Q06,ST42Q07,ST42Q08,ST42Q09,ST42Q10,ST43Q01,ST43Q02,ST43Q03,ST43Q04,ST43Q05,ST43Q06,ST44Q01,ST44Q03,ST44Q04,ST44Q05,ST44Q07,ST44Q08,ST46Q01,ST46Q02,ST46Q03,ST46Q04,ST46Q05,ST46Q06,ST46Q07,ST46Q08,ST46Q09,ST48Q01,ST48Q02,ST48Q03,ST48Q04,ST48Q05,ST49Q01,ST49Q02,ST49Q03,ST49Q04,ST49Q05,ST49Q06,ST49Q07,ST49Q09,ST91Q01,ST91Q02,ST91Q03,ST91Q04,ST91Q05,ST91Q06,ST93Q01,ST93Q03,ST93Q04,ST93Q06,ST93Q07,ST94Q05,ST94Q06,ST94Q09,ST94Q10,ST94Q14,ST96Q01,ST96Q02,ST96Q03,ST96Q05,ANXMAT,FAILMAT,INTMAT,MATBEH,MATHEFF,MATINTFC,MATWKETH,SCMAT,USEMATH,PV1MATH,PV2MATH,PV3MATH,PV4MATH,PV5MATH


### The PISA2012 main datafile

In [25]:
### set up a variable that points to the csv file 
pisa = "pisa2012.csv"

In [26]:
### take a look at the ‘head’ of the csv file to see what the contents might look like
pd.read_csv(pisa, nrows=5)

,Unnamed: 0,CNT,SUBNATIO,STRATUM,OECD,NC,SCHOOLID,STIDSTD,ST01Q01,ST02Q01,...,W_FSTR75,W_FSTR76,W_FSTR77,W_FSTR78,W_FSTR79,W_FSTR80,WVARSTRR,VAR_UNIT,SENWGT_STU,VER_STU
0,1,Albania,80000,ALB0006,Non-OECD,Albania,1,1,10,1,...,13.7954,13.9235,13.1249,13.1249,4.3389,13.0829,19,1,0.2098,22NOV13
1,2,Albania,80000,ALB0006,Non-OECD,Albania,1,2,10,1,...,13.7954,13.9235,13.1249,13.1249,4.3389,13.0829,19,1,0.2098,22NOV13
2,3,Albania,80000,ALB0006,Non-OECD,Albania,1,3,9,1,...,12.7307,12.7307,12.7307,12.7307,4.2436,12.7307,19,1,0.1999,22NOV13
3,4,Albania,80000,ALB0006,Non-OECD,Albania,1,4,9,1,...,12.7307,12.7307,12.7307,12.7307,4.2436,12.7307,19,1,0.1999,22NOV13
4,5,Albania,80000,ALB0006,Non-OECD,Albania,1,5,9,1,...,12.7307,12.7307,12.7307,12.7307,4.2436,12.7307,19,1,0.1999,22NOV13


In [27]:
### create a local sqllite database
csv_dbase = create_engine('sqlite:///csv_dbase.db')

In [28]:
### iterate through the CSV file in chunks and store the data into sqllite

chunksize = 10000
i = 0
j = 1
for df in pd.read_csv(pisa, chunksize=chunksize, 
                      encoding='iso-8859-1', iterator=True, low_memory=False):
      df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) 
      df.index += j
      i+=1
      df.to_sql('table', csv_dbase, if_exists='append')
      j = df.index[-1] + 1

In [29]:
###create the cleaned dataframe that contains the 'selected_codes' only 
df_pisa = pd.read_sql('SELECT CNT,SUBNATIO,STRATUM,OECD,STIDSTD,\
                            ST01Q01,ST04Q01,\
                            ST35Q01,ST35Q02,ST35Q03,ST35Q04,ST35Q05,ST35Q06,\
                            ST42Q01,ST42Q02,ST42Q03,ST42Q04,ST42Q05,\
                            ST42Q06,ST42Q07,ST42Q08,ST42Q09,ST42Q10,\
                            ST43Q01,ST43Q02,ST43Q03,ST43Q04,ST43Q05,ST43Q06,\
                            ST44Q01,ST44Q03,ST44Q04,ST44Q05,ST44Q07,ST44Q08,\
                            ST46Q01,ST46Q02,ST46Q03,ST46Q04,ST46Q05,ST46Q06,\
                            ST46Q07,ST46Q08,ST46Q09,\
                            ST48Q01,ST48Q02,ST48Q03,ST48Q04,ST48Q05,\
                            ST49Q01,ST49Q02,ST49Q03,ST49Q04,ST49Q05, \
                            ST49Q06,ST49Q07,ST49Q09,\
                            ST91Q01,ST91Q02,ST91Q03,ST91Q04,ST91Q05,ST91Q06,\
                            ST93Q01,ST93Q03,ST93Q04,ST93Q06,ST93Q07,\
                            ST94Q05,ST94Q06,ST94Q09,ST94Q10, ST94Q14,\
                            ST96Q01,ST96Q02,ST96Q03,ST96Q05,\
                            ANXMAT,FAILMAT,INTMAT,MATBEH,MATHEFF,\
                            MATINTFC,MATWKETH,SCMAT,USEMATH,\
                            PV1MATH,PV2MATH,PV3MATH,PV4MATH,PV5MATH \
                            FROM "table"', csv_dbase)

In [30]:
df_pisa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485490 entries, 0 to 485489
Data columns (total 91 columns):
CNT         485490 non-null object
SUBNATIO    485490 non-null int64
STRATUM     485490 non-null object
OECD        485490 non-null object
STIDSTD     485490 non-null int64
ST01Q01     485490 non-null int64
ST04Q01     485490 non-null object
ST35Q01     315860 non-null object
ST35Q02     315315 non-null object
ST35Q03     314873 non-null object
ST35Q04     315160 non-null object
ST35Q05     314843 non-null object
ST35Q06     313389 non-null object
ST42Q01     313855 non-null object
ST42Q02     313502 non-null object
ST42Q03     312176 non-null object
ST42Q04     311980 non-null object
ST42Q05     312624 non-null object
ST42Q06     312327 non-null object
ST42Q07     312583 non-null object
ST42Q08     312456 non-null object
ST42Q09     312223 non-null object
ST42Q10     312853 non-null object
ST43Q01     314971 non-null object
ST43Q02     314182 non-null object
ST43Q03     31349

In [31]:
### list of participating countries
set(df_pisa.CNT)

{'Albania',
 'Argentina',
 'Australia',
 'Austria',
 'Belgium',
 'Brazil',
 'Bulgaria',
 'Canada',
 'Chile',
 'China-Shanghai',
 'Chinese Taipei',
 'Colombia',
 'Connecticut (USA)',
 'Costa Rica',
 'Croatia',
 'Czech Republic',
 'Denmark',
 'Estonia',
 'Finland',
 'Florida (USA)',
 'France',
 'Germany',
 'Greece',
 'Hong Kong-China',
 'Hungary',
 'Iceland',
 'Indonesia',
 'Ireland',
 'Israel',
 'Italy',
 'Japan',
 'Jordan',
 'Kazakhstan',
 'Korea',
 'Latvia',
 'Liechtenstein',
 'Lithuania',
 'Luxembourg',
 'Macao-China',
 'Malaysia',
 'Massachusetts (USA)',
 'Mexico',
 'Montenegro',
 'Netherlands',
 'New Zealand',
 'Norway',
 'Perm(Russian Federation)',
 'Peru',
 'Poland',
 'Portugal',
 'Qatar',
 'Romania',
 'Russian Federation',
 'Serbia',
 'Singapore',
 'Slovak Republic',
 'Slovenia',
 'Spain',
 'Sweden',
 'Switzerland',
 'Thailand',
 'Tunisia',
 'Turkey',
 'United Arab Emirates',
 'United Kingdom',
 'United States of America',
 'Uruguay',
 'Vietnam'}

In [32]:
### replace 'Florida (USA)', 'Connecticut (USA)' and 'Massacusets (USA)' 
### with 'United States of America
df_pisa['CNT'].replace('Connecticut (USA)', 'United States of America', inplace=True)
df_pisa['CNT'].replace('Florida (USA)', 'United States of America', inplace=True)
df_pisa['CNT'].replace('Massachusetts (USA)', 'United States of America', inplace=True)


In [33]:
### replace 'Perm(Russian Federation)' with 'Russian Federation'
df_pisa['CNT'].replace('Perm(Russian Federation)', 'Russian Federation', inplace=True)

In [34]:
### combine 'China-Shangai', 'Hong King - China', 'Macao-China'  as 'China'
df_pisa['CNT'].replace('China-Shanghai', 'China', inplace=True)
df_pisa['CNT'].replace('Hong Kong-China', 'China', inplace=True)
df_pisa['CNT'].replace('Macao-China', 'China', inplace=True)

In [35]:
### the updated list of countries
set(df_pisa.CNT)

{'Albania',
 'Argentina',
 'Australia',
 'Austria',
 'Belgium',
 'Brazil',
 'Bulgaria',
 'Canada',
 'Chile',
 'China',
 'Chinese Taipei',
 'Colombia',
 'Costa Rica',
 'Croatia',
 'Czech Republic',
 'Denmark',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Hungary',
 'Iceland',
 'Indonesia',
 'Ireland',
 'Israel',
 'Italy',
 'Japan',
 'Jordan',
 'Kazakhstan',
 'Korea',
 'Latvia',
 'Liechtenstein',
 'Lithuania',
 'Luxembourg',
 'Malaysia',
 'Mexico',
 'Montenegro',
 'Netherlands',
 'New Zealand',
 'Norway',
 'Peru',
 'Poland',
 'Portugal',
 'Qatar',
 'Romania',
 'Russian Federation',
 'Serbia',
 'Singapore',
 'Slovak Republic',
 'Slovenia',
 'Spain',
 'Sweden',
 'Switzerland',
 'Thailand',
 'Tunisia',
 'Turkey',
 'United Arab Emirates',
 'United Kingdom',
 'United States of America',
 'Uruguay',
 'Vietnam'}

In [36]:
### the number of participating countries, as defined here
len(set(df_pisa.CNT))

62

In [37]:
### review the cleaned dataframe
df_pisa.head(4)

,CNT,SUBNATIO,STRATUM,OECD,STIDSTD,ST01Q01,ST04Q01,ST35Q01,ST35Q02,ST35Q03,...,MATHEFF,MATINTFC,MATWKETH,SCMAT,USEMATH,PV1MATH,PV2MATH,PV3MATH,PV4MATH,PV5MATH
0,Albania,80000,ALB0006,Non-OECD,1,10,Female,Disagree,Agree,Disagree,...,-0.77,-0.7332,0.2882,0.41,NaN,406.8469,376.4683,344.5319,321.1637,381.9209
1,Albania,80000,ALB0006,Non-OECD,2,10,Female,Strongly agree,Strongly agree,Disagree,...,0.34,-0.2514,0.6490,NaN,NaN,486.1427,464.3325,453.4273,472.9008,476.0165
2,Albania,80000,ALB0006,Non-OECD,3,9,Female,Strongly agree,Strongly agree,Agree,...,0.34,-0.2514,2.0389,NaN,NaN,533.2684,481.0796,489.6479,490.4269,533.2684
3,Albania,80000,ALB0006,Non-OECD,4,9,Female,None,None,None,...,NaN,NaN,NaN,0.18,NaN,412.2215,498.6836,415.3373,466.7472,454.2842


In [38]:
### get an overall description of the numerical data
df_pisa.describe()

,SUBNATIO,STIDSTD,ST01Q01,ANXMAT,FAILMAT,INTMAT,MATBEH,MATHEFF,MATINTFC,MATWKETH,SCMAT,USEMATH,PV1MATH,PV2MATH,PV3MATH,PV4MATH,PV5MATH
count,4.854900e+05,485490.000000,485490.000000,314764.000000,314448.000000,316708.000000,313847.000000,315948.000000,301360.000000,314501.000000,314607.000000,290260.000000,485490.000000,485490.000000,485490.000000,485490.000000,485490.000000
mean,4.315457e+06,6134.066201,9.813323,0.152647,-0.013110,0.212424,0.241209,-0.046626,-0.012782,0.135775,0.035656,0.067043,469.621653,469.648358,469.648930,469.641832,469.695396
std,2.524434e+06,6733.144944,3.734726,0.955031,1.029037,1.004716,1.054971,0.973588,0.997417,1.009700,0.955625,1.031781,103.265391,103.382077,103.407631,103.392286,103.419170
min,8.000000e+04,1.000000,7.000000,-2.370000,-3.766600,-1.780000,-2.140200,-3.750000,-1.532900,-3.450300,-2.180000,-0.774900,19.792800,6.473000,42.226200,24.622200,37.085200
25%,2.030000e+06,1811.000000,9.000000,-0.470000,-0.530000,-0.340000,-0.456700,-0.630000,-0.733200,-0.401700,-0.520000,-0.774900,395.318600,395.318600,395.240700,395.396500,395.240700
50%,4.100000e+06,3740.000000,10.000000,0.060000,-0.076000,0.300000,0.217100,-0.180000,-0.138100,0.109900,-0.060000,-0.774900,466.201900,466.124000,466.201900,466.279800,466.435600
75%,6.880000e+06,7456.000000,10.000000,0.790000,0.640000,0.910000,0.811000,0.540000,0.658400,0.649000,0.650000,0.869500,541.057800,541.447300,541.291500,541.447300,541.447300
max,8.580000e+06,33806.000000,96.000000,2.550000,3.906700,2.290000,4.424900,2.270000,1.456500,2.716700,2.260000,2.801100,962.229300,957.010400,935.745400,943.456900,907.625800


In [39]:
### store the cleaned dataframe as a csv file
df_pisa.to_csv('pisa2012_clean.csv', index=False)